# 毫米波雷达配置参数合理性校验
1. 主要校验毫米波雷达需要的cfg文件及DCA采集板需要的cf.json文件的参数配置是否正确。
2. 参数的约束条件来自于AWR2243自身的器件特性，具体请参考AWR2243数据手册、mmwave SDK用户手册、chirp编程手册。
3. 若参数满足约束条件将以青色输出调试信息，若不满足则以紫色或黄色输出。
4. 需要注意的是，本程序约束条件并非完全准确，故特殊情况下即使参数全都满足约束条件，也有概率无法正常运行。

### 载入相关库

In [1]:
from color_log.clog import ColorLog, MODE_CONSOLE_LOG  # pip install py-color-log
import math
import json
from mmwave.dataloader import DCA1000
log = ColorLog(name='radar_param_logger', mode=MODE_CONSOLE_LOG)

### 读取配置文件
配置文件信息既可以通过DCA1000.read_config()方法从cfg文件读取，也可以手动输入ADC_PARAMS字典，其结构请参考修改配置文件章节。

In [2]:
# 读取毫米波雷达cfg配置文件
_,_,ADC_PARAMS,CFG_PARAMS=DCA1000.AWR2243_read_config("configFiles/AWR2243_mmwaveconfig.txt")
print(ADC_PARAMS)
print(CFG_PARAMS)

# 读取DCA采集板cf.json配置文件
with open('configFiles/cf.json') as cf_json:
    cf_json_load = json.loads(cf_json.read())
    print(cf_json_load)

{'chirps': 128, 'rx': 4, 'tx': 3, 'samples': 256, 'IQ': 2, 'bytes': 2, 'startFreq': 76.99999997019768, 'idleTime': 7.0, 'adc_valid_start_time': 7.0, 'rampEndTime': 25.0, 'freq_slope': 99.98738765716553, 'txStartTime': 1.0, 'sample_rate': 15000, 'frame_periodicity': 20.0}
{'txAntMask': 7, 'numTxChan': 3, 'rxAntMask': 15, 'numRxChan': 4, 'lvdsBW': 600, 'numlaneEn': 4}
{'DCA1000Config': {'dataLoggingMode': 'raw', 'dataTransferMode': 'LVDSCapture', 'dataCaptureMode': 'ethernetStream', 'lvdsMode': 1, 'dataFormatMode': 3, 'packetDelay_us': 5}}


### 计算约束条件
若参数满足约束条件将以青色输出调试信息，若不满足则以紫色或黄色输出。以黑色输出的为计算出的距离、速度分辨率等信息。

In [3]:
# 约束条件主要参考Programming Chirp Parameters in TI Radar Devices (Rev. A).pdf

# 计算线性调频chirp脉冲的斜率是否满足约束条件：freqSlope<=266MHz/us
if (ADC_PARAMS['startFreq'] >= 76):
    CLI_FREQ_SCALE_FACTOR = 3.6 # 77GHz
else:
    CLI_FREQ_SCALE_FACTOR = 2.7 # 60GHz
mmwStartFreqConst = math.trunc(ADC_PARAMS['startFreq'] * (1<<26) / CLI_FREQ_SCALE_FACTOR )
startFreqConst_actual = mmwStartFreqConst * (CLI_FREQ_SCALE_FACTOR / (1<<26))
log.debug('mmwStartFreqConst: %d,startFreqConst_actual: %f(GHz)'%(mmwStartFreqConst,startFreqConst_actual))
mmwFreqSlopeConst = math.trunc(ADC_PARAMS['freq_slope'] * (1<<26) / (CLI_FREQ_SCALE_FACTOR*1e3*900) )
freqSlopeConst_actual = mmwFreqSlopeConst * ((CLI_FREQ_SCALE_FACTOR*1e3*900) / (1<<26))
logstr='mmwFreqSlopeConst: %d,freqSlopeConst_actual: %f(MHz/us) must <= 266'%(mmwFreqSlopeConst,freqSlopeConst_actual)
if(freqSlopeConst_actual<=266.0215):
    log.info(logstr)
else:
    log.error(logstr)

# 计算帧重复周期是否满足约束条件：300(us)<=frame_periodicity<=1.342(s)（参考DFP手册）
mmwFramePeriodicityConst = math.trunc(ADC_PARAMS['frame_periodicity'] / (5*1e-6) )
framePeriodicityConst_actual = mmwFramePeriodicityConst * 5*1e-6
logstr=('mmwFramePeriodicityConst: %d, 0.3(ms) <= frame periodicity: %.3f(ms) <= 1342(ms)'%(mmwFramePeriodicityConst,framePeriodicityConst_actual))
if(0.3<=framePeriodicityConst_actual and framePeriodicityConst_actual<=1342):
    log.info(logstr)
else:
    log.error(logstr)

# 计算活动帧周期（Tf=N*Tc）是否满足约束条件：active_frame_time<framePeriodicityConst_actual
chirp_repetition_period=ADC_PARAMS['tx']*(ADC_PARAMS['idleTime']+ADC_PARAMS['rampEndTime'])# Tc（us），chirp脉冲间隔周期
log.debug('chirp repetition period:%.2f(us)->max single tx chirp freq:%.2f(kHz)'%(chirp_repetition_period,1e3/chirp_repetition_period))
active_frame_time = ADC_PARAMS['chirps']*chirp_repetition_period/1e3 # Tf=N*Tc（ms）
logstr='active frame time: %.3f(ms) must < frame periodicity: %.3f(ms)'%(active_frame_time,framePeriodicityConst_actual)
if(active_frame_time<framePeriodicityConst_actual):
    log.info(logstr)
    all_tx_chirp_freq=ADC_PARAMS['tx']*ADC_PARAMS['chirps']/framePeriodicityConst_actual
    log.debug('All tx chirp repetition period:%.2f(us)->All tx chirp freq:%.2f(kHz)'%(1e3/all_tx_chirp_freq,all_tx_chirp_freq))
else:
    log.error(logstr)

# 计算采样带宽是否满足约束条件：采样带宽<有效脉冲斜坡带宽（由chirp脉冲斜坡持续时间及斜率决定）
adc_sample_time = 1e3*ADC_PARAMS['samples']/ADC_PARAMS['sample_rate']
max_adc_sample_time = ADC_PARAMS['rampEndTime']-ADC_PARAMS['adc_valid_start_time']-0.15 #不减0.15会造成末尾采样点失真
log.debug('adc start time:%.2f(us) adc sample time:%.2f(us) max adc sample time:%.2f(us) ramp end time:%.2f(us)'%(ADC_PARAMS['adc_valid_start_time'],adc_sample_time,max_adc_sample_time,ADC_PARAMS['rampEndTime']))
adc_sample_sweep_bandwidth = freqSlopeConst_actual * 1e6 * adc_sample_time
max_adc_sweep_bandwidth = freqSlopeConst_actual * 1e6 * max_adc_sample_time
logstr=('adc_sample_sweep_bandwidth:%.2f(MHz) must < max_adc_sweep_bandwidth:%.2f(MHz)'%(adc_sample_sweep_bandwidth/1e6,max_adc_sweep_bandwidth/1e6))
if(adc_sample_sweep_bandwidth < max_adc_sweep_bandwidth):
    log.info(logstr)
else:
    log.error(logstr)

# 计算总脉冲斜坡带宽是否满足约束条件：总脉冲斜坡带宽<4GHz
max_sweep_bandwidth=(81-startFreqConst_actual)*1e9
total_ramp_sweep_bandwidth = freqSlopeConst_actual * 1e6 * ADC_PARAMS['rampEndTime']
logstr=('total_ramp_sweep_bandwidth:%.2f(MHz) must <= %.0f(MHz)'%(total_ramp_sweep_bandwidth/1e6,max_sweep_bandwidth/1e6))
if(total_ramp_sweep_bandwidth/1e6<=max_sweep_bandwidth/1e6):
    log.info(logstr)
else:
    log.error(logstr)

# 计算两脉冲间隔空闲时间是否满足约束条件：ADC_PARAMS['idleTime']>=synthesizerRampDownTime
if(total_ramp_sweep_bandwidth/1e6<1000):
    synthesizerRampDownTime=2
elif(total_ramp_sweep_bandwidth/1e6<2000):
    synthesizerRampDownTime=3.5
elif(total_ramp_sweep_bandwidth/1e6<3000):
    synthesizerRampDownTime=5
else:
    synthesizerRampDownTime=7
logstr='idle time: %.3f(us) must >= Synthesizer Ramp Down Time: %.3f(us)'%(ADC_PARAMS['idleTime'],synthesizerRampDownTime)
if(ADC_PARAMS['idleTime']>=synthesizerRampDownTime):
    log.info(logstr)
else:
    log.error(logstr)

# 计算并显示距离、速度分辨率等参数
midFreq=startFreqConst_actual*1e9+(ADC_PARAMS['adc_valid_start_time']+adc_sample_time/2)*freqSlopeConst_actual*1e6
midFreqWaveLenth=3e8/midFreq

numDopplerBins = 2**math.ceil(math.log2(ADC_PARAMS['chirps']))
numRangeBins = 2**math.ceil(math.log2(ADC_PARAMS['samples']))

rangeResolutionMeters = 3e8 / (2 * adc_sample_sweep_bandwidth)
rangeIdxToMeters = (3e8 * ADC_PARAMS['sample_rate'] * 1e3) / (2 * freqSlopeConst_actual * 1e12 * numRangeBins)
dopplerResolutionMps = midFreqWaveLenth / (2 * numDopplerBins * chirp_repetition_period*1e-6)
maxRange = (300 * 0.8 * ADC_PARAMS['sample_rate']) / (2 * freqSlopeConst_actual * 1e3)
# maxRange = 0.8 * rangeResolutionMeters * ADC_PARAMS['samples']
maxVelocity = midFreqWaveLenth / (4 * chirp_repetition_period*1e-6)

log.debug('max range:%.2f(m)'%maxRange)
log.debug('range resolution:%.4f(m),range interbin resolution:%.4f(m)'%(rangeResolutionMeters,rangeIdxToMeters))
log.debug('max velocity:%.2f(m/s)'%maxVelocity)
log.debug('velocity resolution:%.2f(m/s)'%(dopplerResolutionMps))

# 计算混频输出的最大拍频信号IF带宽是否满足约束条件：maximum_beat_frequency<=20(MHz)
maximum_beat_frequency = 2e6*freqSlopeConst_actual*maxRange/3e8
logstr=('max beat frequency:%.2f(MHz) must <= max I/F bandwith 20(MHz)'%maximum_beat_frequency)
if(maximum_beat_frequency<=20):
    log.info(logstr)
else:
    log.error(logstr)

# 计算采样率是否满足约束条件：maximum_sampling_frequency<=22.5(MHz)
logstr=('sample rate:%.2f(MHz) must <= max sampling frequency 22.5(MHz)'%(ADC_PARAMS['sample_rate']/1e3))
if(ADC_PARAMS['sample_rate']/1e3<=22.5):
    log.info(logstr)
else:
    log.error(logstr)

# 计算片内DSP需要的radar cube内存大小是否满足约束条件：radar_cube_size<=1024(KB)
# radar_cube_size = numRangeBins*ADC_PARAMS['chirps']*ADC_PARAMS['tx']*ADC_PARAMS['rx']*4/1024
# logstr=('radar cube size:%.2f(KB) must <= 1024(KB)'%radar_cube_size)
# if(radar_cube_size<=1024):
#     log.info(logstr)
# else:
#     log.error(logstr)

# 计算原始IQ数据量是否满足约束条件：LVDS Data Size Per Chirp <= max Send Bytes Per Chirp
LVDSDataSizePerChirp = ADC_PARAMS['samples']*ADC_PARAMS['rx']*ADC_PARAMS['IQ']*ADC_PARAMS['bytes']+52
LVDSDataSizePerChirp = math.ceil(LVDSDataSizePerChirp/256)*256
maxSendBytesPerChirp = (ADC_PARAMS['idleTime']+ADC_PARAMS['rampEndTime'])*CFG_PARAMS['numlaneEn']*CFG_PARAMS['lvdsBW']/8
logstr=("LVDS Data Size Per Chirp:%d(Bytes) must <= max Send Bytes Per Chirp:%d(Bytes)"%(LVDSDataSizePerChirp,maxSendBytesPerChirp))
if(LVDSDataSizePerChirp<=maxSendBytesPerChirp):
    log.info(logstr)
else:
    log.error(logstr)

# 计算FPGA吞吐量（与延时有关）是否满足约束条件：FPGA Packet Delay <= min required Packet Delay
BYTES_IN_PACKET = 1456  # Data in payload per packet from FPGA
BYTES_OF_PACKET = 1466  # payload size per packet from FPGA
UDP_PACKET_OVERHEAD = 8
IP_OVERHEAD     = 20
ETH_OVERHEAD    = 14
SizeInMBperSec = ADC_PARAMS['samples']*ADC_PARAMS['rx']*ADC_PARAMS['IQ']*ADC_PARAMS['bytes']*ADC_PARAMS['tx']*ADC_PARAMS['chirps']*(1e-3/framePeriodicityConst_actual)
overheadRatio = (BYTES_OF_PACKET+UDP_PACKET_OVERHEAD+IP_OVERHEAD+ETH_OVERHEAD)/BYTES_IN_PACKET
FPGA_throughput = 8*SizeInMBperSec*overheadRatio
log.debug('FPGA throughput:%.2f(Mbps)'%(FPGA_throughput))
minPacketDelay = 138.57 * math.exp(-FPGA_throughput / 179.157) + 2.73 # 根据DCA1000EVA手册数据拟合
logstr=("FPGA Packet Delay:%d(us) should <= min required Packet Delay:%.2f(us)"%(cf_json_load['DCA1000Config']['packetDelay_us'],minPacketDelay))
if(cf_json_load['DCA1000Config']['packetDelay_us']<=minPacketDelay):
    log.info(logstr)
else:
    log.warn(logstr)

print('current params:',ADC_PARAMS)

2023-04-08 23:48:03 [DEBUG] - mmwStartFreqConst: 1435384035,startFreqConst_actual: 77.000000(GHz)
2023-04-08 23:48:03 [INFO] - mmwFreqSlopeConst: 2071,freqSlopeConst_actual: 99.987388(MHz/us) must <= 266
2023-04-08 23:48:03 [INFO] - mmwFramePeriodicityConst: 4000000, 0.3(ms) <= frame periodicity: 20.000(ms) <= 1342(ms)
2023-04-08 23:48:03 [DEBUG] - chirp repetition period:96.00(us)->max single tx chirp freq:10.42(kHz)
2023-04-08 23:48:03 [INFO] - active frame time: 12.288(ms) must < frame periodicity: 20.000(ms)
2023-04-08 23:48:03 [DEBUG] - All tx chirp repetition period:52.08(us)->All tx chirp freq:19.20(kHz)
2023-04-08 23:48:03 [DEBUG] - adc start time:7.00(us) adc sample time:17.07(us) max adc sample time:17.85(us) ramp end time:25.00(us)
2023-04-08 23:48:03 [INFO] - adc_sample_sweep_bandwidth:1706.45(MHz) must < max_adc_sweep_bandwidth:1784.77(MHz)
2023-04-08 23:48:03 [INFO] - total_ramp_sweep_bandwidth:2499.68(MHz) must <= 4000(MHz)
2023-04-08 23:48:03 [INFO] - idle time: 7.000(u

current params: {'chirps': 128, 'rx': 4, 'tx': 3, 'samples': 256, 'IQ': 2, 'bytes': 2, 'startFreq': 76.99999997019768, 'idleTime': 7.0, 'adc_valid_start_time': 7.0, 'rampEndTime': 25.0, 'freq_slope': 99.98738765716553, 'txStartTime': 1.0, 'sample_rate': 15000, 'frame_periodicity': 20.0}


### 修改配置文件参数
可以在此处试着修改一些参数并重新运行“计算约束条件”程序块校验参数的合理性

In [32]:
#在这里修改参数后直接运行上边的程序块看效果
ADC_PARAMS={
    'chirps': 128,               # 对应txt里的loopCount
    'rx': 4, 
    'tx': 3, 
    'samples': 256,              # 对应txt里的numAdcSamples，rlFrameCfg_t里的为rlProfileCfg_t里的两倍
    'IQ': 2, 
    'bytes': 2, 
    'startFreq': 76,             # 对应txt里的startFreqConst，记得转为mmwStartFreqConst
    'idleTime': 7.0,             # 对应txt里的idleTimeConst，记得转为10ns为单位（idleTimeConst=700，相当于7us）
    'adc_valid_start_time': 4.0, # 对应txt里的adcStartTimeConst，记得转为10ns为单位（adcStartTimeConst=400，相当于4us）
    'rampEndTime': 15.53,        # 对应txt里的rampEndTime，记得转为10ns为单位（rampEndTime=1553，相当于15.53us）
    'freq_slope': 265.008,       # 对应txt里的freqSlopeConst，记得转为mmwFreqSlopeConst
    'txStartTime': 1,            # 对应txt里的txStartTime，记得转为10ns为单位（txStartTime=100，相当于1us）
    'sample_rate': 22500,        # 对应txt里的digOutSampleRate
    'frame_periodicity': 8.952   # 对应txt里的periodicity，记得转为mmwFramePeriodicityConst
}

### 测试
测试ColorLog模块的功能

In [1]:
from color_log.clog import ColorLog
log = ColorLog(name='radar_param_logger', mode=MODE_CONSOLE_LOG)

log.debug('这是 DEBUG 日志，白色')
log.info('这是 INFO 日志，青绿')
log.warn('这是 WARN 日志，黄色')
log.error('这是 ERROR 日志，紫色')
try :
    a = 1 / 0
except :
    log.critical('这是 CRITICAL 日志，红色')

2022-07-02 13:36:52 [DEBUG] - 这是 DEBUG 日志，白色
2022-07-02 13:36:52 [INFO] - 这是 INFO 日志，青绿
2022-07-02 13:36:52 [WARNING] - 这是 WARN 日志，黄色
2022-07-02 13:36:52 [ERROR] - 这是 ERROR 日志，紫色
2022-07-02 13:36:52 [CRITICAL] - 这是 CRITICAL 日志，红色
2022-07-02 13:36:52 [CRITICAL] - Traceback (most recent call last):
  File "C:\Users\gwf\AppData\Local\Temp\ipykernel_14708\2271216170.py", line 8, in <cell line: 7>
    a = 1 / 0
ZeroDivisionError: division by zero

